# Load SELEX Matricies and Run MOODS from Python

We're going to load the SELEX matricies from Regulation into Python, and output them as nicely formatted JSON. Then we'll select the first 50 matricies, and use this to scan some enhancer sequences (also loaded into Python, not using the file system).

In [1]:
import os
import itertools
import csv
import random
import json
from multiprocessing import Pool
from pathlib import Path
from tfbs.pfm_reader import pfm_reader, PFM
from tfbs.moods import PWM, Scanner, iter_fasta


In [2]:
SELEX = Path("/home/malcolm/Data/Regulation/PWMs/SELEX")

# build an iterator that makes everything look like one big file
def load_pfms(d: Path):
    for pfm_file in d.iterdir():
        with open(pfm_file) as f:
            yield [pfm_file.stem, *f.readlines()]


pfm_iter = pfm_reader(itertools.chain.from_iterable(load_pfms(SELEX / "matrices")))

# read in SELEX PFMs and index by file name
SELEX_pfms = [PFM(id=info[0], PFM=pfm) for info, pfm in pfm_iter]

SELEX_pfms[50].dict()


{'id': 'NR1I3_KL_TTTAGC40NGAC_AGTTCANNNNAGTTCA_m1_c3b1_elife2014',
 'PFM': [[248, 5, 8, 74, 12, 248, 46, 57, 70, 70, 248, 0, 12, 61, 26, 248],
  [16, 17, 0, 55, 248, 12, 75, 63, 46, 41, 23, 7, 1, 42, 248, 30],
  [92, 248, 127, 158, 34, 174, 57, 91, 61, 64, 99, 248, 92, 101, 24, 183],
  [11, 5, 248, 248, 94, 30, 70, 36, 70, 72, 2, 4, 248, 248, 81, 30]],
 'metadata': {}}

From here we can serialize everything as JSON:

In [3]:
with open(SELEX / "selex.json", 'w') as jsonfile:
    json.dump([pfm.dict() for pfm in SELEX_pfms], jsonfile, indent=4)

And then read it back in:

In [4]:
with open(SELEX / "selex.json") as jsonfile:
    pfms = [PFM(**item) for item in json.load(jsonfile)]

pfms[50].dict()

{'id': 'NR1I3_KL_TTTAGC40NGAC_AGTTCANNNNAGTTCA_m1_c3b1_elife2014',
 'PFM': [[248, 5, 8, 74, 12, 248, 46, 57, 70, 70, 248, 0, 12, 61, 26, 248],
  [16, 17, 0, 55, 248, 12, 75, 63, 46, 41, 23, 7, 1, 42, 248, 30],
  [92, 248, 127, 158, 34, 174, 57, 91, 61, 64, 99, 248, 92, 101, 24, 183],
  [11, 5, 248, 248, 94, 30, 70, 36, 70, 72, 2, 4, 248, 248, 81, 30]],
 'metadata': {}}

Now we can select the PWMs we want to scan, in this case 50 random PWMs from the SELEX data:

In [5]:
chosen_pfms = random.choices(SELEX_pfms, k=50)


In [6]:
PWMs = [PWM(p.PFM, p.id, threshold=("pvalue", 0.0001)) for p in chosen_pfms]

Build the Scanner object:

In [7]:
s = Scanner(PWMs)

Read in the fasta sequences:

In [8]:
seqs = iter_fasta(Path.home() / "Projects/Enhancers/pwms/fasta/ENCFF503GCK_chr11.fa")

def scan(fa: tuple[str, str]): return s.scan(fa)

with Pool(16) as p:
    results = list(p.map(scan, seqs))


Now we have a list of hits for each sequence:

In [9]:
results[:10]

[{'header': 'chr11:87214:87280:ID11.100666', 'hits': []},
 {'header': 'chr11:87240:87307:ID11.100675', 'hits': []},
 {'header': 'chr11:108018:108077:ID11.10073',
  'hits': [Hit(TF='HOXB2_PITX1_AY_TGGCCT40NTAT_TAATKRNGGATTA_m1_c3', start=32, end=45, score=7.967856907398772, strand='+')]},
 {'header': 'chr11:108060:108095:ID11.10078', 'hits': []},
 {'header': 'chr11:110913:111017:ID11.10083',
  'hits': [Hit(TF='TEAD4_HOXB13_AY_TCTATG40NTAG_RRAATGCARTAAA_m1_c3', start=62, end=75, score=6.840922442303286, strand='-')]},
 {'header': 'chr11:128422:128507:ID11.100883',
  'hits': [Hit(TF='MEIS1_MAX_AT_TATCTG40NTCG_NTGACRNNNNNNCACGTGN_m1_c3', start=58, end=77, score=5.386541986282942, strand='-')]},
 {'header': 'chr11:130594:130654:ID11.100888', 'hits': []},
 {'header': 'chr11:131180:131340:ID11.100893',
  'hits': [Hit(TF='ETV2_SREBF2_AAA_TAAAGT40NAGC_RTMRCGTGACGGAWGN_m1_c2', start=82, end=98, score=6.435413149516675, strand='-'),
   Hit(TF='HOXB2_ETV7_AY_TGAGGT40NTCC_NSCGGAARNNNNNNMATTAN_m1_c3

We can output or store this however we want, in this case we can convert to chromosomal co-ordinates in a BED-like file:

NB I have no idea if the MOODS output is 0 or 1 based, we will have to test this.

In [10]:
header = ["chr", "start", "end", "name", "score", "strand", "PWM"]

print("\t".join(header))

for r in results[:50]:
    chrom, start, end, name = r['header'].split(':')
    for h in r['hits']:
        h_start = int(start) + h.start
        h_end = int(start) + h.end
        print("\t".join(map(str, [chrom, h_start, h_end, name, h.score, h.strand, h.TF])))

chr	start	end	name	score	strand	PWM
chr11	108050	108063	ID11.10073	7.967856907398772	+	HOXB2_PITX1_AY_TGGCCT40NTAT_TAATKRNGGATTA_m1_c3
chr11	110975	110988	ID11.10083	6.840922442303286	-	TEAD4_HOXB13_AY_TCTATG40NTAG_RRAATGCARTAAA_m1_c3
chr11	128480	128499	ID11.100883	5.386541986282942	-	MEIS1_MAX_AT_TATCTG40NTCG_NTGACRNNNNNNCACGTGN_m1_c3
chr11	131262	131278	ID11.100893	6.435413149516675	-	ETV2_SREBF2_AAA_TAAAGT40NAGC_RTMRCGTGACGGAWGN_m1_c2
chr11	131236	131256	ID11.100893	6.308213098663654	-	HOXB2_ETV7_AY_TGAGGT40NTCC_NSCGGAARNNNNNNMATTAN_m1_c3
chr11	132599	132621	ID11.100898	7.258471089670726	+	CUX1_SOX15_AY_TGCTTC40NTTA_NATCRATNNNNNNNAACAATRS_m1_c3b0
chr11	146846	146861	ID11.10107	7.347840094955376	-	FOXJ2_HOXB13_AAF_TACGCA40NAAT_NTTTATNRNTMAACA_m1_c2
chr11	146805	146821	ID11.10107	9.356341186301758	-	MGA_DLX2_AAD_TGCGGT40NTCA_SYAATTANWGGTGYGA_m2_c2u
chr11	175109	175137	ID11.101262	9.614079968208154	+	TEAD4_FOXI1_AX_TACATC40NATA_RCATWCCNNNNNNNNNNNNNNRTMAACA_m1_c3b1
chr11	175417	175431	